![](measure_CF_calls_logo_v1.0.png)

# measure horseshoe bat calls
A package to automate measurements of Horseshoe bat calls - (or any similar sound!). The package is still under development - I would greatly appreciate any kind of feedback on how/whether it works on your data!

### What is a horseshoe bat call - why have I never heard one before?
Don't know - no worries! Haven't heard one before, no surprise - they're typically ultrasonic!
However, if you really want to imagine what it'd be like if you could hear them, this is what a horseshoe bat call would sound like. You can do this by slowing down the horseshoe bat calls by a *lot* (10X)!

In [1]:
from IPython.display import Audio, display
Audio(filename='slowed_down_call_2018-08-17_34_211.wav')

So, as you may have heard in the playback - the horseshoe bat call sounds like a whistle with a 'rising' part, a constant part and a 'descending' part. These are technically known as the rising frequency modulation (FM), constant frequency (CF) and the descending FM parts of a horseshoe bat call. Not all calls have both rising and descending, and some calls may have only the rising FM or the descending FM. The relative durations of the Cf and FM are also known to change depending on the behavioural context at hand. When plotted on a spectrogram, people have described the call to resemble a table (side-profile) or a staple-pin - and this is where the logo of this package comes from!


### What does this package do?
It's all in the name. It measures various properties of a horseshoe bat call. 

Given a call, the package segments out CF and FM components and calculates various measures. The FM and CF parts are separated in the waveform and various measurements are made on them including:

* CF peak frequency
* FM end frequency/ies
* CF and FM durations
* CF and FM rms and energy

### Who is this package useful for?
Mainly those studying bat calls, and in specific those analysing horseshoe bat calls. However, there may perhaps be a whole host of sounds which looks similar -- and so it might also be useful for the analysis of any sound that looks like a bat CF call!

### Why did you develop this package?
Measuring the sub-structure of horseshoe bat calls can be tiresome and error-prone when done manually. While the human eye can recognise the CF and FM parts of the call very well, segmenting it consistently is hard to do. This is because we rely on a spectrographic representation - the settings of which again influences how a call is segmented into FM and CF. Even existing automated methods in the field rely on spectrographic representations.  Moreover, to my knowledge, there aren't any  open-source tools developed specifically to measre call parameters for horseshoe bat calls. 


### Acknowledgements
The example data is a subset of audio recorded of *Rhinolophus mehelyi* and *Rhinolophus euryale* bats as they flew in and out of an evening roosting spot. Data collection was done by Neetash MR and Thejasvi Beleyur. Audio data annotation and selection done by Aditya Krishna. 

I thank Neetash MR, Diana Schoeppler & Holger R Goerlitz for their inputs in the development of this package.

### How do I use this package?

### Installation: 
This package works with both Python 2 and 3. 
Download the package using pip with:

```pip install measure_horseshoebat_calls ```

or

download this repo from Github,open up a terminal window, navigate to the folder,  and type:

```python setup.py install ```

### Usage:

The measure_horseshoe_bat_calls package takes in an audio file snippet with a *single* horseshoe bat call in it and outputs various measures of this call. Let us first load a manually selected part of a recording with a single horseshoe bat call in it. 

In [2]:
%matplotlib notebook

In [3]:
import measure_horseshoe_bat_calls
from  measure_horseshoe_bat_calls import measure_a_horseshoe_bat_call as measure
from measure_horseshoe_bat_calls import segment_horseshoebat_call as segment
from measure_horseshoe_bat_calls.view_horseshoebat_call import *
from measure_horseshoe_bat_calls import data

import soundfile as sf 
import scipy.signal as signal

In [4]:
# load a part of an audio file known to have a horseshoe bat call 
chosen_audio = data.example_calls[25]
fs = data.fs

# view the waveform + spectrogram
visualise_call(chosen_audio, fs)

<IPython.core.display.Javascript object>

(<matplotlib.axes._subplots.AxesSubplot at 0x11ea6708>,
 <matplotlib.axes._subplots.AxesSubplot at 0x14f3ca88>)

### Segmenting the call from the silent background. 
Measuring temporal features such as call, CF and FM durations relies on an accurate and tight window selection that includes the call without too much of the background. Having more 'background' in the analyses will lead to inaccurate measurements of all parameters eg. duration and rms. Selecting a tight window can be simply done by hand, but also yields inconsistent results each time. Automating this process allows reproducibility across time. 

In [5]:
# get a tight window which only has the call. 
main_call_window = segment.segment_call_from_background(chosen_audio, fs, 
                                                                          background_threshold=-16)

In [ ]:
waveform, specgram = check_call_background_segmentation(chosen_audio, fs, main_call_window)
# choose a tighter ylim to see the call better
specgram.set_ylim(50000, 125000)

<IPython.core.display.Javascript object>

### Segmenting a horseshoe bat call into the FM and CF components:
To actually quantitatively characterise a horseshoe bat call we first need to define the constant frequency (CF) and frequency modulated (FM) parts of it. 

In [ ]:
# choose only the call without any background
only_call = chosen_audio[main_call_window]

In [ ]:
cf, fm, info = segment.segment_call_into_cf_fm(only_call, fs, percentage_peak=0.99)

In [ ]:
wave, spec = check_call_parts_segmentation(only_call, fs, cf, fm)
spec.set_ylim(50000, 120000)

### Segmenting the call into CF and FM parts


In [ ]:
call_parts, measurements = measure.measure_hbc_call(only_call, fs, cf, fm)

In [ ]:
spec = make_overview_figure(only_call, fs, measurements)
spec.set_ylim(70000, 125000)

### Understanding the measurements and overview figure:
The plot above shows start and stop times of the FMs in red. The blue lines along the FM indicate the lowest frequency (at -X dB threshold) and are drawn over the duration of the entire FM segment. The blue line in the CF is drawn at the call peak frequency and is drawn over the duration of the CF portion. 

In [ ]:
measurements

### The call parts : CF and FM 
The output sounds are the audio corresponding to the CF segmentation and FM segmentation. The call parts are output as a list with two objects in them. The first is the CF sound, and the second is another list with one or two FM sounds in it. 

In [ ]:
show_all_call_parts(only_call, call_parts, fs)


### Choosing the right parameter settings for *your* bat calls!
The segmentation is of course affected by the parameter settings given to ```measure_hbc_call ```. The current default set may be good for many situations - but you may want better! 

### How does the call segmentation work?
The call segmentation works on the fact that horseshoe bat calls have a relatively stereotyped structure. The CF part of a call is typically flanked by one or two FMs. ```segment_call_into_cf_fm ``` goes through the following steps for every input audio. 

* 1. Separate call from background by running a continuous wavelet transform on the audio. The coefficients of all scales with centre frequencies in the signal range are summed up. This forms a kind of signal-band 'energy-profile' which tracks the onset and offset of the signal with high temporal resolution. 

* 2. Separate FM and CF by forming two versions of the call filtered at the same threshold frequency: i) low-pass filtered ii) high-pass filtered. The threshold frequency for low and high pass filtering is user-definable, and set by default to 0.99 of the detected peak frequency.

* 3. A dB rms profile of the low and high pass filtered audio is calculated. The main idea is that the low pass dBrms profile will have peaks towards the edges (where the FM sweeps are), while the highpass dBrms profile will have a broad bump in the centre. How to separate out the two reliably? The separation is done by taking a difference profile. eg. The portions of the $lowpass_{dB \:rms}-highpass_{dB \:rms}$ difference profile which are $\geq 1$ are the FM parts, while the portions of the $highpass_{dB \:rms}-lowpass_{dB \:rms} >0 $ are the CF parts.


### Feedback and Contributions Welcome!! 

### Author: Thejasvi Beleyur,   Acoustic and Functional Ecology Group, Max Planck Institute for Ornithology, Seewiesen

-- Last updated 2020-03-12 15:50 CET